# Final Project Sequence 3

This sequence is about detecting and tracking pedestrians, cyclists and cars. In the sequence is an occlusion which the code needs to try and track when the objects goes behind that. This should be obtained based on YOLO object detection and the kalman filter we was teached about in lecture 7 about state estimation. This sequence does not have any ground truth so here we cant validate how well the model performs but here we test how good of a mdoel we obtained in sequence 2 so this is the test sequence.

# Step 1
Start by look at the raw sequence

In [1]:
import cv2
import glob
import os
from ultralytics import YOLO 
import numpy as np
import math
import time
from scipy.optimize import linear_sum_assignment
import pandas as pd
from tqdm import tqdm

# Change this path if needed
img_folder = "34759_final_project_rect/seq_03/image_02/data"

images = sorted(glob.glob(os.path.join(img_folder, "*.png")))
print(f"Found {len(images)} images in {img_folder}")

if not images:
    raise FileNotFoundError("No images found. Check your folder path.")

for i, img_path in enumerate(images):
    frame = cv2.imread(img_path)
    if frame is None:
        print("Failed to load:", img_path)
        continue

    cv2.imshow("Left Camera Sequence", frame)
    key = cv2.waitKey(30)
    if key == 27:  # ESC
        break

cv2.destroyAllWindows()

Found 286 images in 34759_final_project_rect/seq_03/image_02/data


# Step 2
Setup the helper functions and the kalman filter 

In [6]:
# Here is the helper functions that is needed for the yolo model with the kalman filter
def iou(box1, box2):
    # This function is to see how much two bounding boxes is overlapping
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    # The small value is to make sure the system does not get an error of devision by 0
    return inter_area / (union_area + 1e-9) 

def xyxy_to_cxcywh(box):
    # This function turns the bounding box from corner positions into center position, width and height
    w = box[2] - box[0]
    h = box[3] - box[1]
    cx = box[0] + w/2
    cy = box[1] + h/2
    return np.array([cx, cy, w, h])

def cxcywh_to_xyxy(cx, cy, w, h):
    # This function turns the bounding box from center position, width and height into corner positions 
    x1 = cx - w/2
    y1 = cy - h/2
    x2 = cx + w/2
    y2 = cy + h/2
    return [x1, y1, x2, y2] 

def associate_tracks_dets(tracks, dets_xyxy, dets_cls, iou_thresh=0.55):
    # This functions matches existing object tracks with new detections to see if the new detections matches the existing one
    if len(tracks) == 0:
        return [], [], list(range(len(dets_xyxy)))

    # IoU between tracktions and detections
    iou_matrix = np.zeros((len(tracks), len(dets_xyxy)), dtype=np.float32)
    for t, tr in enumerate(tracks):
        t_box = tr.get_xyxy()
        for d, d_box in enumerate(dets_xyxy):
            if tr.cls != dets_cls[d]:
                iou_matrix[t, d] = 0.0
            else:
                iou_matrix[t, d] = iou_xyxy(t_box, d_box)
                
    matches = []
    unmatched_tracks = set(range(len(tracks)))
    unmatched_dets = set(range(len(dets_xyxy)))

    # Make matching based on IoU
    if iou_matrix.size > 0:
        indices = np.dstack(np.unravel_index(np.argsort(-iou_matrix.ravel()), iou_matrix.shape))[0]
        for t, d in indices:
            if iou_matrix[t, d] < iou_thresh:
                break
            if t in unmatched_tracks and d in unmatched_dets:
                matches.append((t, d))
                unmatched_tracks.remove(t)
                unmatched_dets.remove(d)
                
    return matches, list(unmatched_tracks), list(unmatched_dets)

In [7]:
class KalmanBoxTracker:
    # This class is the kalman filter for tracking the bounding boxes
    def __init__(self, init_z, cls, dt, init_velocity=None):
        #This function is for initializing the kalman filter
        
        self.dt = float(dt)

        #This is the state vector
        self.x = np.zeros((8,1), dtype=np.float32)
        
        self.x[0:4,0] = init_z.reshape(4) # initialize position and size
        if init_velocity is not None:
            self.x[4:8,0] = np.reshape(init_velocity, (4,)) # initialize velocity
        
        self.F = np.eye(8, dtype=np.float32) # State transition matrix
        for i in range(4): 
            self.F[i, i+4] = self.dt
            
        # Measurement matrix    
        self.H = np.zeros((4,8), dtype=np.float32)
        self.H[0,0] = self.H[1,1] = self.H[2,2] = self.H[3,3] = 1.0

        # Uncertainty Covariance matrix
        self.P = np.eye(8, dtype=np.float32) * 20.0 

        # Process noise 
        self.Q = np.eye(8, dtype=np.float32)
        self.Q[0,0]=0.5; self.Q[1,1]=0.5 # cx, cy
        self.Q[2,2]=0.1; self.Q[3,3]=0.1 # w, h
        self.Q[4,4]=4.0; self.Q[5,5]=4.0 # vx, vy
        self.Q[6,6]=1.0; self.Q[7,7]=1.0 # vm, vh

        # Measurement noise
        self.R = np.eye(4, dtype=np.float32) * 2.0
        self.R[2,2] = 10.0 # Increased noise for width measurement
        self.R[3,3] = 10.0 # Increased noise for height measurement

        # Identity matrix
        self.I = np.eye(8, dtype=np.float32)

        # Initialization of the needed data
        self.cls = cls # object class
        self.id = None # ID of the tracked object
        self.age = 0 # The number of frames since first detection
        self.miss = 0 # The number of frames where it is not seen
        self.score = 0.0 # The confidence of the detection
        self.hits = 0 # The number of times where it has been hit
        self.confirmed = False # Confirmation of the tracktion
        self.last_detection_box = cxcywh_to_xyxy(self.x[0,0], self.x[1,0], self.x[2,0], self.x[3,0])
        self.last_update_frame = -1

    def predict(self):
        # This function predict the next state 
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        self.age += 1

        # Reset if the detection is missed
        if self.miss > 0:
            self.x[6, 0] = 0.0 # vw (velocity width)
            self.x[7, 0] = 0.0 # vh (velocity height)
            
        return self.get_xyxy()

    def update(self, z, score, frame_idx=None):
        # This is the update function of the kalman filter
        z = z.reshape(4,1)
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + (K @ y)
        self.P = (self.I - K @ self.H) @ self.P
        
        self.miss = 0
        self.score = float(score)
        self.hits += 1
        if self.hits >= 3:
            self.confirmed = True
        
        self.last_detection_box = cxcywh_to_xyxy(float(self.x[0,0]), float(self.x[1,0]), 
                                                 float(self.x[2,0]), float(self.x[3,0]))
        self.last_update_frame = frame_idx if frame_idx is not None else self.last_update_frame

    def get_xyxy(self):
        # This function is to get the current bounding boxes
        cx = float(self.x[0,0]); cy = float(self.x[1,0])
        w = max(1.0, float(self.x[2,0])); h = max(1.0, float(self.x[3,0]))
        
        # This is to avoid unrealistic bounding boxes
        max_ratio = 10.0 
        
        if w > h * max_ratio: w = h * max_ratio
        if h > w * max_ratio: h = w * max_ratio
            
        return np.array(cxcywh_to_xyxy(cx, cy, w, h), dtype=np.float32)


In [10]:
## Here is the config for the object detection and tracking using both yolo and kalman filter
IMG_FOLDER = "34759_final_project_rect/seq_03/image_02/data"
OUTPUT_TRACK_FILE = "inference_results_seq03.csv"
MODEL_PATH = "yolo11l.pt" 

FPS = 10.0 
DT = 1.0 / FPS
IOU_MATCH_THRESH = 0.55     
MAX_INIT_PREV_DIST_SQ = 400.0**2
SCORE_DECAY = 0.99
DETECTION_CONF_THRESHOLD = 0.5 
MAX_MISS_CONFIRMED = 150 
MAX_MISS_UNCONFIRMED = 5 

# Start the yolo + kalman tracking
def run_2d_tracker():
    try:
        model = YOLO(MODEL_PATH)
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return

    allowed_classes = {"person", "car", "bicycle"}
    tracks = []
    next_id = 1
    prev_frame_preds = [] 
    tracked_results = [] 

    images = sorted(glob.glob(os.path.join(IMG_FOLDER, "*.png")))
    if not images:
        print(f"Error: No images found in {IMG_FOLDER}. Check path.")
        return

    print(f"Starting 2D tracking on {len(images)} frames...")

    for frame_id, frame_path in enumerate(tqdm(images, desc="Tracking Frames")):
        
        # Read the frame
        frame = cv2.imread(frame_path)
        if frame is None:
            continue

        frame_for_yolo = frame.copy() # clean input to YOLO
        frame_disp = frame.copy() # used only for drawing 

        # Start the yolo detection
        results = model(frame_for_yolo, verbose=False)
        raw = []
        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy() # Extracts the boundingbox coordinates into a numoy array
            confs = r.boxes.conf.cpu().numpy() # Extracsts the confidence of the detections
            clss = r.boxes.cls.cpu().numpy() # Extracts the class label for the detections

            for box, conf, cls in zip(boxes, confs, clss):
                label = model.names[int(cls)]
                if label in allowed_classes and conf >= DETECTION_CONF_THRESHOLD: # Remove objects detected under a confidence of 50%
                    raw.append((box, conf, label))

        # Merge cyclists which is a person on a bike
        used = set()
        preds = []
    
        for i, (box1, conf1, label1) in enumerate(raw):
            if label1 != "person":
                continue
    
            for j, (box2, conf2, label2) in enumerate(raw):
                if label2 != "bicycle" or j in used:
                    continue
    
                if iou(box1, box2) > 0.2: #Threshold for when it is a cyclist
                    x1 = min(int(box1[0]), int(box2[0]))
                    y1 = min(int(box1[1]), int(box2[1]))
                    x2 = max(int(box1[2]), int(box2[2]))
                    y2 = max(int(box1[3]), int(box2[3]))
                    # Add cyclists to the detections
                    preds.append(([x1, y1, x2, y2], (conf1 + conf2)/2, "cyclist"))
                    used.add(i)
                    used.add(j)

        # Add the other detections
        for k, (box, conf, label) in enumerate(raw):
            if k in used: continue
            if label == "car":
                preds.append((list(map(int, box)), conf, "car"))
            elif label == "person":
                preds.append((list(map(int, box)), conf, "pedestrian"))

        # Start the kalman prediction
        for tr in tracks: tr.predict()

        # Match the detections to the tracking 
        dets_xyxy = [np.array(b, dtype=np.float32) for b, c, l in preds]
        dets_cls = [l for b, c, l in preds]
        dets_z = [xyxy_to_cxcywh(b) for b, c, l in preds]
        matches, unmatched_tracks, unmatched_dets = associate_tracks_dets(tracks, dets_xyxy, dets_cls, IOU_MATCH_THRESH)

        # Start the update with the kalman filter
        for t, d in matches:
            tracks[t].update(dets_z[d], preds[d][1], frame_id)
            tracks[t].cls = dets_cls[d]

        # Remove the matches that is not correct
        for t in unmatched_tracks:
            tr = tracks[t]
            tr.miss += 1
            tr.score *= SCORE_DECAY
            
            max_miss_allowed = MAX_MISS_CONFIRMED if tr.confirmed else MAX_MISS_UNCONFIRMED
            
            if tr.miss > max_miss_allowed or tr.score < 0.05:
                tracks[t] = None

        tracks = [t for t in tracks if t is not None]

        # Start new tracking
        prev_by_cls = {}
        for p in prev_frame_preds: prev_by_cls.setdefault(p["cls"], []).append(p)

        for d in unmatched_dets:
            box = dets_xyxy[d]
            # Remove dublicates
            if any(iou_xyxy(t.get_xyxy(), box) > 0.5 for t in tracks): continue
            
            init_vel = None
            cls = dets_cls[d]
            if cls in prev_by_cls:
                cx, cy, _, _ = dets_z[d]
                # Estimate the velocity
                best = min(prev_by_cls[cls], key=lambda p: (cx-p["cx"])**2 + (cy-p["cy"])**2, default=None)
                if best:
                    d2 = (cx-best["cx"])**2 + (cy-best["cy"])**2
                    if d2 < MAX_INIT_PREV_DIST_SQ:
                        init_vel = [(cx-best["cx"])/DT, (cy-best["cy"])/DT, 0.0, 0.0]

            # Create the new tracktion
            tr = KalmanBoxTracker(dets_z[d], cls, DT, init_vel)
            tr.id = next_id; next_id += 1
            tr.score = preds[d][1]
            tracks.append(tr)

        # Remove the overlapping tracktions
        to_rem = set()
        for i in range(len(tracks)):
            if i in to_rem: continue
            for j in range(i+1, len(tracks)):
                if j in to_rem: continue
                if iou_xyxy(tracks[i].get_xyxy(), tracks[j].get_xyxy()) > 0.4:
                    if tracks[i].confirmed != tracks[j].confirmed:
                        to_rem.add(j if tracks[i].confirmed else i)
                    else:
                        to_rem.add(j if tracks[i].score > tracks[j].score else i)
        tracks = [t for k, t in enumerate(tracks) if k not in to_rem]

        #Start the visualization and save the data
        for tr in tracks:
            if not tr.confirmed and tr.miss > 0: continue
            if tr.score < 0.5: continue 

            box = tr.get_xyxy().astype(int)
            is_active = tr.miss == 0
            
            color = (0, 255, 0) if tr.confirmed and is_active else (255, 255, 0)
            
            label = f"ID:{tr.id} {tr.cls[:3]} H:{tr.hits} S:{tr.score:.2f}"

            # Save the data
            tracked_results.append({
                'frame': frame_id,
                'id': tr.id,
                'class': tr.cls,
                'score': tr.score,
                'left': box[0], 'top': box[1], 'right': box[2], 'bottom': box[3],
                'status': "Confirmed" if tr.confirmed else "New"
            })

            cv2.rectangle(frame_disp, (box[0], box[1]), (box[2], box[3]), color, 2)
            cv2.putText(frame_disp, label, (box[0], box[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

        prev_frame_preds = []
        for box, conf, cls in preds:
            z = xyxy_to_cxcywh(np.array(box))
            prev_frame_preds.append({"cx": z[0], "cy": z[1], "w": z[2], "h": z[3], "cls": cls})

        cv2.imshow("2D Tracker (YOLO Detections)", frame_disp)
        if cv2.waitKey(1) == 27: break

    cv2.destroyAllWindows()

    # Save the output
    if tracked_results:
        results_df = pd.DataFrame(tracked_results)
        results_df.to_csv(OUTPUT_TRACK_FILE, index=False)
        print(f" 2D Tracking results saved to {OUTPUT_TRACK_FILE}.")
    else:
        print(" No detections were tracked. Output file not created.")

if __name__ == "__main__":
    run_2d_tracker()

Starting 2D tracking on 286 frames...


Tracking Frames: 100%|███████████████████████████████████████████████████████████████| 286/286 [02:21<00:00,  2.03it/s]

 2D Tracking results saved to inference_results_seq03.csv.


# Stereo vision

Load the camera calibraation, compute the disparity and estimate 3D coordinates

In [11]:
# Load files (Change paths if needed)
left_folder = "34759_final_project_rect/seq_03/image_02/data/"
right_folder = "34759_final_project_rect/seq_03/image_03/data/"
calib_path = "34759_final_project_rect/calib_cam_to_cam.txt"

def load_projection_matrices(calib_file):
    # Load the projection from the calibration file
    P = {}
    with open(calib_file, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("P_rect_"):
                key, values = line.split(":")
                cam = int(key.split("_")[-1])
                P[cam] = np.array(list(map(float, values.split()))).reshape(3,4)
    return P

P = load_projection_matrices(calib_path)
P_left = P[2]   # P_rect_02
P_right = P[3]  # P_rect_03

f = P_left[0,0]               # focale in pixel
B = abs(P_right[0,3] / f)     # baseline = Tx / f
print(f"Focale: {f:.2f} px, Baseline: {B:.3f} m")

np.set_printoptions(precision=15, suppress=False)
print("P_left (02):\n", P_left)
print("P_right (03):\n", P_right)

def compute_disparity_map(left_img, right_img):
    # Compute the disparity map between the left and right image
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    
    # SGBM parameters
    min_disp = 0
    num_disp = 64 
    block_size = 9
    
    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disp, 
        numDisparities=num_disp, 
        blockSize=block_size, 
        P1=8 * 3 * block_size**2,
        P2=32 * 3 * block_size**2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32
    )
    
    disparity = stereo.compute(left_gray, right_gray).astype(np.float32) / 16.0
    disparity[disparity < 0] = np.nan 
    
    return disparity

def estimate_depth_from_disparity(disparity_map, bbox):
    # Estimate the depth Z
    left, top, right, bottom = [int(x) for x in bbox]
    
    h, w = disparity_map.shape
    top = max(0, top); bottom = min(h, bottom)
    left = max(0, left); right = min(w, right)
    
    disp_region = disparity_map[top:bottom, left:right]
    valid_disparities = disp_region[~np.isnan(disp_region)]
    
    if len(valid_disparities) < 10:
        return -1.0 
        
    median_disparity = np.median(valid_disparities)
    
    if median_disparity <= 1e-6:
        return -1.0 
        
    Z_pred = BASELINE_X_FX / median_disparity
    
    return Z_pred

def project_3d_to_2d(X_c, Y_c, Z_c, FX, FY, CX, CY):
    #Projects a 3D point (X, Y, Z) to 2D image coordinates (u, v)
    if Z_c <= 0: return None, None
        
    u = int((X_c * FX / Z_c) + CX)
    v = int((Y_c * FY / Z_c) + CY)
    
    return u, v


def calculate_pred_3d_bottom_center(track_row, Z_pred):
    # Calculates the 3D center corresponding to the 2D bounding box bottom
    if Z_pred <= 0:
        return None
        
    # 2D bottom center
    u_c = (track_row["left"] + track_row["right"]) / 2 # horizontal center
    v_bottom = track_row["bottom"]                     # vertical bottom edge
    
    X_pred = Z_pred * (u_c - CX) / FX
    
    Y_pred = Z_pred * (v_bottom - CY) / FY
    
    return [X_pred, Y_pred, Z_pred]

Focale: 707.05 px, Baseline: 0.473 m
P_left (02):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02  4.575831e+01]
 [ 0.000000e+00  7.070493e+02  1.805066e+02 -3.454157e-01]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  4.981016e-03]]
P_right (03):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02 -3.341081e+02]
 [ 0.000000e+00  7.070493e+02  1.805066e+02  2.330660e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  3.201153e-03]]


In [12]:
# Path for import of images, the tracktion and the calibration
IMG_FOLDER_LEFT = "34759_final_project_rect/seq_03/image_02/data"
IMG_FOLDER_RIGHT = "34759_final_project_rect/seq_03/image_03/data" 
TRACK_FILE_PATH = "inference_results_seq03.csv" # Input from the 2D tracking script
CALIB_FILE_PATH = "34759_final_project_rect/calib_cam_to_cam.txt"

# Set the global parameters
FX = 707.0493
FY = 707.0493
CX = 604.0814
CY = 180.5066
BASELINE_X_FX = 45.75831 

# Visualization Colors
COLOR_PREDICTED_BOX = (255, 0, 0)      # Blue box for 2D track
COLOR_PRED_3D_POINT = (0, 0, 255)      # Red point for Predicted 3D center (BOTTOM)

# Make the visualization with the 3D position
def run_3d_visualization():
    try:
        # Load Predicted Tracks data
        df_tracks = pd.read_csv(TRACK_FILE_PATH)
        df_tracks = df_tracks.rename(columns={'Frame': 'frame'}) # Fix for potential case mismatch
        tracks_grouped = df_tracks.groupby('frame')
                
    except FileNotFoundError:
        print(f"Error: Track file not found at {TRACK_FILE_PATH}. Run the 2D tracker first.")
        return
    except KeyError:
        print(f"Error: Missing 'frame' or 'Frame' column in {TRACK_FILE_PATH}. Check file content.")
        return


    images_left = sorted(glob.glob(os.path.join(IMG_FOLDER_LEFT, "*.png")))
    images_right = sorted(glob.glob(os.path.join(IMG_FOLDER_RIGHT, "*.png")))

    if not images_left or len(images_left) != len(images_right):
        print("Error: Image files missing or count mismatch. Check IMG_FOLDER_LEFT/RIGHT paths.")
        return

    print(f" Loaded {len(images_left)} frames. Generating 3D visualization...")

    for frame_id, img_path_left in enumerate(tqdm(images_left, desc="Visualizing Frames")):
        img_path_right = images_right[frame_id]
        
        frame_left = cv2.imread(img_path_left)
        frame_right = cv2.imread(img_path_right)
        
        if frame_left is None or frame_right is None:
            continue

        frame_disp = frame_left.copy()
        
        # Compute Disparity Map
        disparity_map = compute_disparity_map(frame_left, frame_right)

        # Draw Predicted 2D Boxes and Predicted 3D BOTTOM Centers (Blue Box & Red Dot)
        if frame_id in tracks_grouped.groups:
            frame_tracks = tracks_grouped.get_group(frame_id)
            
            for _, track_row in frame_tracks.iterrows():
                if not all(col in track_row for col in ["left", "top", "right", "bottom", "id", "class"]):
                    continue

                bbox = [track_row["left"], track_row["top"], track_row["right"], track_row["bottom"]]
                track_id = int(track_row['id'])
                cls = track_row['class']
                
                # Depth estimation
                Z_pred_val = estimate_depth_from_disparity(disparity_map, bbox)
                
                # Estimate the 3D center corresponding to the 2D bottom edge
                pred_3d_center = calculate_pred_3d_bottom_center(track_row, Z_pred_val)
                
                # Draw the 2D Bounding Box
                box_int = [int(x) for x in bbox]
                cv2.rectangle(frame_disp, (box_int[0], box_int[1]), (box_int[2], box_int[3]), COLOR_PREDICTED_BOX, 2)
                
                box_label = f"P:{cls} ID:{track_id}"
                cv2.putText(frame_disp, box_label, (box_int[0], box_int[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PREDICTED_BOX, 1)

                # Draw the Green Predicted 3D Center (BOTTOM)
                if pred_3d_center is not None:
                    X_pred, Y_pred, Z_pred = pred_3d_center
                    
                    # Project Predicted 3D center to 2D image coordinates
                    u_pred, v_pred = project_3d_to_2d(X_pred, Y_pred, Z_pred, FX, FY, CX, CY)
                    
                    if u_pred is not None:
                        # Draw Green Predicted dot
                        cv2.circle(frame_disp, (u_pred, v_pred), 5, COLOR_PRED_3D_POINT, -1)
                        
                        # Label with X, Y, Z position (in meters)
                        pred_label_line1 = f"3D PRED:"
                        pred_label_line2 = f"X:{X_pred:.1f} Y:{Y_pred:.1f}"
                        pred_label_line3 = f"Z:{Z_pred:.1f} m"
                        
                        # Draw labels
                        cv2.putText(frame_disp, pred_label_line1, (u_pred + 10, v_pred - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED_3D_POINT, 1)
                        cv2.putText(frame_disp, pred_label_line2, (u_pred + 10, v_pred - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED_3D_POINT, 1)
                        cv2.putText(frame_disp, pred_label_line3, (u_pred + 10, v_pred), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED_3D_POINT, 1)

        # Display the frame
        cv2.imshow("3D Tracking Visualization", frame_disp)
        
        if frame_id == 0:
            # Show first frame for 5 seconds (5000 ms)
            key = cv2.waitKey(30000)
        else:
            # Show subsequent frames for 50 ms
            key = cv2.waitKey(50)
        
        if key == 27:
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_3d_visualization()

 Loaded 286 frames. Generating 3D visualization...


Visualizing Frames: 100%|████████████████████████████████████████████████████████████| 286/286 [01:44<00:00,  2.74it/s]


In [13]:
# Path for import of images, the tracktion and the calibration
IMG_FOLDER_LEFT = "C:/Users/senik/Documents/2. Semester/Perception for Autonomous Systems/Final project/34759_final_project_rect/seq_03/image_02/data"
IMG_FOLDER_RIGHT = "C:/Users/senik/Documents/2. Semester/Perception for Autonomous Systems/Final project/34759_final_project_rect/seq_03/image_03/data" 
TRACK_FILE_PATH = "inference_results_seq03.csv" 
CALIB_FILE_PATH = "C:/Users/senik/Documents/2. Semester/Perception for Autonomous Systems/Final project/34759_final_project_rect/calib_cam_to_cam.txt"
OUTPUT_REPORT_FILE = "3d_track_report_seq03.csv" # New output file for your report data

# Function for the data generation
def run_3d_data_generation():
    results = []
    
    try:
        # Load Predicted Tracks data
        df_tracks = pd.read_csv(TRACK_FILE_PATH)
        df_tracks = df_tracks.rename(columns={'Frame': 'frame'}) # Ensure 'frame' column name
        tracks_grouped = df_tracks.groupby('frame')
                
    except FileNotFoundError:
        print(f"Error: Track file not found at {TRACK_FILE_PATH}. Run the 2D tracker first.")
        return
    except KeyError:
        print(f"Error: Missing 'frame' or 'Frame' column in {TRACK_FILE_PATH}. Check file content.")
        return

    images_left = sorted(glob.glob(os.path.join(IMG_FOLDER_LEFT, "*.png")))
    images_right = sorted(glob.glob(os.path.join(IMG_FOLDER_RIGHT, "*.png")))

    if not images_left or len(images_left) != len(images_right):
        print("Error: Image files missing or count mismatch. Check IMG_FOLDER_LEFT/RIGHT paths.")
        return

    print(f" Loaded {len(images_left)} frames. Calculating 3D positions for tracked objects...")

    for frame_id, img_path_left in enumerate(tqdm(images_left, desc="Processing Frames")):
        img_path_right = images_right[frame_id]
        
        frame_left = cv2.imread(img_path_left)
        frame_right = cv2.imread(img_path_right)
        
        if frame_left is None or frame_right is None:
            continue
        
        # Compute Disparity Map once per frame
        disparity_map = compute_disparity_map(frame_left, frame_right)

        # Get Predicted tracks for this frame
        if frame_id in tracks_grouped.groups:
            frame_tracks = tracks_grouped.get_group(frame_id)
            
            # Iterate over ALL predictions in this frame
            for _, track_row in frame_tracks.iterrows():
                
                # Extract 2D Bounding Box
                bbox = [track_row["left"], track_row["top"], track_row["right"], track_row["bottom"]]
                
                # Estimate Depth (Z)
                Z_pred_val = estimate_depth_from_disparity(disparity_map, bbox)
                
                # Calculate 3D Bottom Center (X, Y)
                pred_3d_center = calculate_pred_3d_bottom_center(track_row, Z_pred_val)
                
                if pred_3d_center is not None:
                    X_pred, Y_pred, Z_pred = pred_3d_center
                else:
                    X_pred, Y_pred, Z_pred = None, None, None

                # Store Results
                results.append({
                    "frame": frame_id,
                    "track_id": int(track_row["id"]),
                    "class": track_row["class"],
                    "score": track_row["score"],
                    "status": track_row["status"],
                    "left": track_row["left"],
                    "top": track_row["top"],
                    "right": track_row["right"],
                    "bottom": track_row["bottom"],
                    "X_pred": X_pred,
                    "Y_pred": Y_pred,
                    "Z_pred": Z_pred # This is the crucial depth value
                })

    # Save the output file
    if results:
        df_report = pd.DataFrame(results)
        df_report.to_csv(OUTPUT_REPORT_FILE, index=False)
        print(f"\n 3D Tracking Report saved to **{OUTPUT_REPORT_FILE}** ({len(df_report)} entries).")
        print("\n--- Example Data ---")
        print(df_report.head())
    else:
        print("\n No predicted tracks were found or processed. Report file not created.")

if __name__ == "__main__":
    run_3d_data_generation()

 Loaded 286 frames. Calculating 3D positions for tracked objects...


Processing Frames: 100%|█████████████████████████████████████████████████████████████| 286/286 [00:52<00:00,  5.43it/s]



 3D Tracking Report saved to **3d_track_report_seq03.csv** (1651 entries).

--- Example Data ---
   frame  track_id       class     score status  left  top  right  bottom  \
0      0         1         car  0.935298    New     0  222    287     364   
1      0         2         car  0.790386    New  1076  149   1197     188   
2      0         3  pedestrian  0.789847    New   939  144    987     289   
3      0         4  pedestrian  0.772622    New   574  149    619     308   
4      0         5         car  0.747246    New   447  169    481     196   

     X_pred    Y_pred    Z_pred  
0 -0.505749  0.201488  0.776387  
1  3.424266  0.048194  4.547410  
2  0.563109  0.170216  1.109292  
3 -0.009182  0.154405  0.856296  
4 -1.066551  0.117964  5.383330  
